In [1]:
import unsloth

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
import torch
from transformers import AutoConfig, AutoModelForCausalLM
from MyLLM.CrispyLLM_RoPE2.modeling_crispy_rope import CrispyLLMConfig, CrispyForCausalLM
from transformers import XLMRobertaTokenizer, PreTrainedTokenizerFast
from datasets import load_dataset, concatenate_datasets

# 3. Kayıt (Auto ile kullanabilmek için)
AutoConfig.register("crispy", CrispyLLMConfig)
AutoModelForCausalLM.register(CrispyLLMConfig, CrispyForCausalLM)

In [3]:
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

In [4]:
max_seq_length = 1024  # Choose any! We auto support RoPE Scaling internally!
dtype = torch.bfloat16 # None for auto detection. bfloat16 for Tesla T4, V100, bfloat16 for Ampere+
load_in_4bit = False 
load_in_8bit = False 

In [5]:


# XLM-Roberta tokenizer yükleniyor
#tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")

In [6]:
special_tokens_dict = {
    "bos_token": "<s>",
    "eos_token": "<|eot_id|>",
    "additional_special_tokens":  [
        "<|im_start|>", "<|im_end|>",
        "<|system|>", "<|user|>", "<|assistant|>",
        "<|start_header_id|>", "<|end_header_id|>", "<|eot_id|>"
    ]
}


#tokenizer.add_special_tokens(special_tokens_dict)

In [7]:



#crispy_config = CrispyLLMConfig(attn_implementation="flash_attention_2", use_flash_attention_2=True, vocab_size=len(tokenizer.get_vocab()), n_heads=16, max_seq_len=max_seq_length, hidden_size=64*16, num_hidden_layers=16, dtype="bfloat16")

#crispy_config._attn_implementation_autoset = True  # 👈 Buraya ekliyorsun

#model = AutoModelForCausalLM.from_config(crispy_config)

In [8]:
model_path = "./Crispy-330M-V2-Rope-NewTokenizer-JustLanguage/checkpoint-10850"

model = AutoModelForCausalLM.from_pretrained(model_path ,  
                                            attn_implementation="flash_attention_2",
                                            trust_remote_code=True,
                                            torch_dtype=torch.bfloat16,
                                            device_map="auto"
      ) 
tokenizer = XLMRobertaTokenizer.from_pretrained("./Crispy-330M-V2-Rope-NewTokenizer-JustLanguage")

📦 Loading weights from model.safetensors


In [9]:
# Tokenizer’a yeni token eklediysen bunu yapman gerekir
#model.resize_token_embeddings(len(tokenizer))


In [10]:
# Tokenizer'dan alınan vocab size
tokenizer_vocab_size = len(tokenizer.get_vocab())
print(f"🔤 Tokenizer vocab size: {tokenizer_vocab_size}")

# Modelin token embedding katmanından alınan vocab size
model_embedding_vocab_size = model.embedding.token_embedding.embedding_layer.num_embeddings
print(f"🧠 Model token embedding vocab size: {model_embedding_vocab_size}")

# Modelin lm_head katmanından alınan çıkış boyutu
model_lm_head_vocab_size = model.lm_head.out_features
print(f"🎯 Model lm_head vocab size: {model_lm_head_vocab_size}")

# Hepsi eşleşiyor mu?
if tokenizer_vocab_size == model_embedding_vocab_size == model_lm_head_vocab_size:
    print("✅ Tokenizer ve model vocab boyutları uyumlu.")
else:
    print("⚠️ UYARI: Vocab size değerleri eşleşmiyor!")


🔤 Tokenizer vocab size: 250010
🧠 Model token embedding vocab size: 250010
🎯 Model lm_head vocab size: 250010
✅ Tokenizer ve model vocab boyutları uyumlu.


In [11]:
model

CrispyForCausalLM(
  (embedding): EmbeddingLayer(
    (token_embedding): TokenEmbedding(
      (embedding_layer): Embedding(250010, 1024)
    )
  )
  (decoderBlocks): ModuleList(
    (0-15): 16 x DecoderBlock(
      (attention_block): AttentionBlock(
        (qkv_proj): Linear(in_features=1024, out_features=3072, bias=True)
        (o_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (rms_norm1): RMSNormBlock(
          (rmsNorm): RMSNorm((1024,), eps=1e-05, elementwise_affine=True)
        )
        (attn): FlashAttentionBlockBase()
        (rope): RotaryPositionalEmbedding()
      )
      (feedforward_network): FeedforwardNetwork(
        (ln1): Linear(in_features=1024, out_features=4096, bias=True)
        (swiglu): SwiGLU(
          (linear1): Linear(in_features=4096, out_features=2048, bias=True)
          (linear2): Linear(in_features=1024, out_features=4096, bias=True)
        )
        (ln2): Linear(in_features=4096, out_features=1024, bias=True)
      )
   

In [12]:
torch.autograd.set_detect_anomaly(True)  # debug amaçlı


In [13]:
assert tokenizer.pad_token_id == 1, "pad_token_id yanlış!"


In [14]:
model = model.train()

In [15]:
model

CrispyForCausalLM(
  (embedding): EmbeddingLayer(
    (token_embedding): TokenEmbedding(
      (embedding_layer): Embedding(250010, 1024)
    )
  )
  (decoderBlocks): ModuleList(
    (0-15): 16 x DecoderBlock(
      (attention_block): AttentionBlock(
        (qkv_proj): Linear(in_features=1024, out_features=3072, bias=True)
        (o_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (rms_norm1): RMSNormBlock(
          (rmsNorm): RMSNorm((1024,), eps=1e-05, elementwise_affine=True)
        )
        (attn): FlashAttentionBlockBase()
        (rope): RotaryPositionalEmbedding()
      )
      (feedforward_network): FeedforwardNetwork(
        (ln1): Linear(in_features=1024, out_features=4096, bias=True)
        (swiglu): SwiGLU(
          (linear1): Linear(in_features=4096, out_features=2048, bias=True)
          (linear2): Linear(in_features=1024, out_features=4096, bias=True)
        )
        (ln2): Linear(in_features=4096, out_features=1024, bias=True)
      )
   

In [16]:
from datasets import load_dataset, Dataset
from datasets import load_from_disk
import os

processed_path = "/media/hosman/Yedek/Datasets/c4_tr_300k"

if not os.path.exists(processed_path):
    datasetC4 = load_dataset("allenai/c4", "tr", split="train", cache_dir="/media/hosman/Yedek/Datasets/", num_proc=4)
    datasetC4 = datasetC4.shuffle(seed=42)
    datasetC4 = datasetC4.select(range(300000)).remove_columns(['timestamp', 'url'])
    datasetC4.save_to_disk(processed_path)


In [17]:
datasetC4 = load_from_disk("/media/hosman/Yedek/Datasets/c4_tr_300k")
datasetC4

Dataset({
    features: ['text'],
    num_rows: 300000
})

In [18]:
datasetWiki = load_dataset("wikimedia/wikipedia", "20231101.tr",split="train", cache_dir="/media/hosman/Yedek/Datasets/", num_proc=4).shuffle(seed=42).select(range(300000)).remove_columns(['id', 'url', 'title'])
datasetWiki

Dataset({
    features: ['text'],
    num_rows: 300000
})

In [19]:
""" datasetText = load_dataset("yasarefe/turkish-texts-dataset-2", split="train", cache_dir="/media/hosman/Yedek/Datasets/", num_proc=4).shuffle(seed=42)
datasetText """

' datasetText = load_dataset("yasarefe/turkish-texts-dataset-2", split="train", cache_dir="/media/hosman/Yedek/Datasets/", num_proc=4).shuffle(seed=42)\ndatasetText '

In [20]:
""" datasetOscarSmall = load_dataset("nthngdy/oscar-small", "unshuffled_deduplicated_tr", split="train", cache_dir="/media/hosman/Yedek/Datasets/", trust_remote_code=True, num_proc=4).shuffle(seed=42).select(range(300000)).shuffle(seed=42).select(range(300000)).remove_columns(['id'])
datasetOscarSmall """

' datasetOscarSmall = load_dataset("nthngdy/oscar-small", "unshuffled_deduplicated_tr", split="train", cache_dir="/media/hosman/Yedek/Datasets/", trust_remote_code=True, num_proc=4).shuffle(seed=42).select(range(300000)).shuffle(seed=42).select(range(300000)).remove_columns([\'id\'])\ndatasetOscarSmall '

In [21]:
from itertools import islice
from datasets import load_dataset, Dataset, concatenate_datasets, load_from_disk
from tqdm import tqdm
import os

processed_path = "/media/hosman/Yedek/Datasets/oscar_tr_1m"
cache_dir = "/media/hosman/Yedek/Datasets/"
chunk_size = 100_000  # parça büyüklüğü
total_size = 1_000_000

if not os.path.exists(processed_path):
    print("Veri seti indiriliyor ve işleniyor...")

    streamed_dataset = load_dataset(
        "oscar-corpus/OSCAR-2201",
        language="tr",
        split="train",
        streaming=True,
        cache_dir=cache_dir
    )

    iterator = iter(streamed_dataset)
    all_chunks = []

    for i in tqdm(range(0, total_size, chunk_size), desc="Veriler alınıyor"):
        chunk = list(islice(iterator, chunk_size))
        if not chunk:
            break
        hf_chunk = Dataset.from_list(chunk)
        all_chunks.append(hf_chunk)

    # Tüm parçaları birleştir
    full_dataset = concatenate_datasets(all_chunks)

    # Disk'e kaydet
    full_dataset.save_to_disk(processed_path)
    print(f"{total_size} örnek başarıyla kaydedildi: {processed_path}")

else:
    print("Dataset daha önce kayıtlı:", processed_path)
    datasetOscar = load_from_disk(processed_path).shuffle(seed=42).shuffle(seed=21).shuffle(seed=15).select(range(300000)).remove_columns(["id", "meta"])


Dataset daha önce kayıtlı: /media/hosman/Yedek/Datasets/oscar_tr_1m


In [22]:
def replace_empty_with_none(example):
    # 'inputs' sütunundaki boş karakteri None ile değiştirelim
    if example['text'] == "":
        example['text'] = None
    return example

# dataset4'teki 'inputs' sütunundaki boş karakterleri None ile değiştir
datasetC4 = datasetC4.map(replace_empty_with_none)
datasetWiki = datasetWiki.map(replace_empty_with_none)
datasetOscar = datasetOscar.map(replace_empty_with_none)

In [23]:
dataset = concatenate_datasets([datasetC4, datasetWiki, datasetOscar])

In [24]:
dataset = dataset.filter(lambda x: x["text"]!=None)


In [25]:
dataset

Dataset({
    features: ['text'],
    num_rows: 900000
})

In [26]:
print(dataset[5]["text"])

Fen Bilimleri 4. Sınıf Slaytları - Ders sunuları
Fen Bilimleri 4.Sınıf Etkinlikleri
Fen ve Teknoloji 4 - Hammadde Bul
Fen ve Teknoloji 4 - Hammadde Bul Etkinliği - Bu etkinlikte ekrana gelecek ürünlerin nelerden yapılmış olabileceğini tahmin etmenizi ve ürünün ham maddesini keşfetmenizi istiyoruz. Başarılar...
Fen ve Teknoloji 4 - Kemik Türleri
Fen ve Teknoloji 4 - Vücudumuz - Kemik Türleri Etkinliği. Vücudumuzdaki kısa, uzun ve yassı kemiklerin hangileri biliyor musunuz? Peki bilginize güveniyor musunuz? Şimdi deneme zamanı.
Fen ve Teknoloji 4 - Madde Ölçme Etkinliği
Fen ve Teknoloji 4 - Madde Ölçme Etkinliği - İki ayrı bölümden oluşan bu etkinliğimizde katı ve sıcı maddelerin ölçülmesi ile ilgili temel bilgilerimizi tazeleyeceğiz. İlk bölümde boşluklara uygun tanımı seçecek, ikinci bölümde cümlelerdeki boşlukları biz dolduracağız.
Fen ve Teknoloji 4 - İskelet Yapalım
Fen ve Teknoloji 4 - Vücudumuz - Destek Sistemi - İskelet yapalım etkinliği. İskeletin tüm parçaları darmadağın oldu. 

In [27]:
print(tokenizer.tokenize(dataset[5]["text"]))

['▁Fen', '▁Bilim', 'leri', '▁4.', '▁Sınıf', '▁Sla', 'yt', 'ları', '▁-', '▁Der', 's', '▁su', 'nu', 'ları', '▁Fen', '▁Bilim', 'leri', '▁4.', 'S', 'ını', 'f', '▁Etkinlik', 'leri', '▁Fen', '▁ve', '▁Teknoloji', '▁4', '▁-', '▁Hamma', 'dde', '▁Bul', '▁Fen', '▁ve', '▁Teknoloji', '▁4', '▁-', '▁Hamma', 'dde', '▁Bul', '▁Et', 'kin', 'liği', '▁-', '▁Bu', '▁etkinlik', 'te', '▁ekran', 'a', '▁gelecek', '▁ürünleri', 'n', '▁ne', 'lerden', '▁yapılmış', '▁olabileceği', 'ni', '▁tahmin', '▁etme', 'nizi', '▁ve', '▁ürünü', 'n', '▁ham', '▁maddesi', 'ni', '▁keşfe', 't', 'meniz', 'i', '▁istiyoruz', '.', '▁Başarı', 'lar', '...', '▁Fen', '▁ve', '▁Teknoloji', '▁4', '▁-', '▁Kemi', 'k', '▁Tür', 'leri', '▁Fen', '▁ve', '▁Teknoloji', '▁4', '▁-', '▁V', 'ü', 'cu', 'd', 'umuz', '▁-', '▁Kemi', 'k', '▁Tür', 'leri', '▁Et', 'kin', 'liği', '.', '▁V', 'ü', 'cu', 'd', 'umuz', 'daki', '▁kısa', ',', '▁uzun', '▁ve', '▁ya', 's', 'sı', '▁kemi', 'k', 'lerin', '▁hangi', 'leri', '▁biliyor', '▁musunuz', '?', '▁Peki', '▁bilgi', 'nize', '▁g

In [28]:
print(tokenizer.decode(tokenizer.encode(dataset[5]["text"])))

<s> Fen Bilimleri 4. Sınıf Slaytları - Ders sunuları Fen Bilimleri 4.Sınıf Etkinlikleri Fen ve Teknoloji 4 - Hammadde Bul Fen ve Teknoloji 4 - Hammadde Bul Etkinliği - Bu etkinlikte ekrana gelecek ürünlerin nelerden yapılmış olabileceğini tahmin etmenizi ve ürünün ham maddesini keşfetmenizi istiyoruz. Başarılar... Fen ve Teknoloji 4 - Kemik Türleri Fen ve Teknoloji 4 - Vücudumuz - Kemik Türleri Etkinliği. Vücudumuzdaki kısa, uzun ve yassı kemiklerin hangileri biliyor musunuz? Peki bilginize güveniyor musunuz? Şimdi deneme zamanı. Fen ve Teknoloji 4 - Madde Ölçme Etkinliği Fen ve Teknoloji 4 - Madde Ölçme Etkinliği - İki ayrı bölümden oluşan bu etkinliğimizde katı ve sıcı maddelerin ölçülmesi ile ilgili temel bilgilerimizi tazeleyeceğiz. İlk bölümde boşluklara uygun tanımı seçecek, ikinci bölümde cümlelerdeki boşlukları biz dolduracağız. Fen ve Teknoloji 4 - İskelet Yapalım Fen ve Teknoloji 4 - Vücudumuz - Destek Sistemi - İskelet yapalım etkinliği. İskeletin tüm parçaları darmadağın ol

In [29]:
dataset = dataset.filter(lambda x:( len(tokenizer.encode(x["text"])) )<max_seq_length)

In [30]:
dataset = dataset.shuffle(seed=42)
dataset = dataset.shuffle(seed=41)
dataset = dataset.shuffle(seed=40)
dataset = dataset.shuffle(seed=39)

In [31]:
from datasets import DatasetDict

# 1. Veriyi train ve test olarak ayırma
# Örneğin, dataset zaten tek bir büyük veri seti (örneğin "data") içeriyor
# Bunu %80 train ve %20 test olarak bölelim
train_dataset, temp_dataset = dataset.train_test_split(test_size=0.1, seed=42).values()

# 2. Test setini de %50 validation ve %50 test olarak bölelim
val_dataset, test_dataset = temp_dataset.train_test_split(test_size=0.2, seed=42).values()

In [32]:
#model.gradient_checkpointing_enable()
model.use_cache = True

In [33]:
import wandb

wb_c = wandb.init(project="Basic LLM Train", name="Crispy-330M-V2-Rope-NewTokenizer-JustLanguage" , resume="allow", id="q5q5kjiy") #id="a7zeymst",id="ecibz7e4" id="dbaxrwf4"
wb_c.watch(model, log="all")
wandb.notebook.init()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: h-osmankarabulut to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


AttributeError: module 'wandb' has no attribute 'notebook'

In [ ]:
import wandb
import evaluate
import numpy as np
from prettytable import PrettyTable
import torch
import re
from rapidfuzz import fuzz

def exact_match(prediction, reference):
    return prediction.strip().lower() == reference.strip().lower()

def contains_correct_result(prediction, reference):
    try:
        ref_nums = [int(s) for s in re.findall(r"\d+", reference)]
        pred_nums = [int(s) for s in re.findall(r"\d+", prediction)]
        return any(num in pred_nums for num in ref_nums)
    except:
        return False

def fuzzy_match_score(prediction, reference):
    return fuzz.ratio(prediction, reference) / 100.0  # normalize to 0-1

def evaluate_model(model, tokenizer, test_dataset, max_seq_length=256):
    """
    Eğitilmiş modeli test veri kümesi üzerinde değerlendirir ve sonuçları wandb'a loglar.
    
    Parametreler:
    - model: Eğitilmiş dil modeli
    - tokenizer: Modelin tokenizer'ı
    - test_dataset: Test veri kümesi (instruction-output içermeli)
    - max_seq_length: Maksimum yanıt uzunluğu (varsayılan: 256)

    Çıktı:
    - Metin tablosu (PrettyTable ile)
    - wandb logları
    """

    # Değerlendirme metriklerini yükleme
    rouge = evaluate.load("rouge")
    bleu = evaluate.load("bleu")
    meteor = evaluate.load("meteor")
    bertscore = evaluate.load("bertscore")

    predictions = []
    references = []
    exact_matches = []
    correct_results = []
    fuzzy_scores = []

    # Modeli değerlendirme moduna al
    model.eval()

    print("🚀 Model test verisi üzerinde değerlendiriliyor...\n")

    for example in test_dataset:
        input_text = f"### Talimat:\n{example['instruction']}\n\n### Yanıt:\n"
        reference_text = example["output"]

        inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

        with torch.no_grad():
            output_ids = model.generate(**inputs, max_new_tokens=max_seq_length)

        decoded_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        decoded_output = decoded_output.split("### Yanıt")[-1].strip()

        predictions.append(decoded_output)
        references.append(reference_text)

        exact_matches.append(exact_match(decoded_output, reference_text))
        correct_results.append(contains_correct_result(decoded_output, reference_text))
        fuzzy_scores.append(fuzzy_match_score(decoded_output, reference_text))

    # Metrik hesaplamaları
    rouge_scores = rouge.compute(predictions=predictions, references=references)
    bleu_score = bleu.compute(predictions=predictions, references=[[ref] for ref in references])
    meteor_score = meteor.compute(predictions=predictions, references=references)
    bert_scores = bertscore.compute(predictions=predictions, references=references, lang="tr")

    bert_precision = np.mean(bert_scores["precision"])
    bert_recall = np.mean(bert_scores["recall"])
    bert_f1 = np.mean(bert_scores["f1"])
    exact_match_score = np.mean(exact_matches)
    correct_result_score = np.mean(correct_results)
    fuzzy_match_avg = np.mean(fuzzy_scores)

    # Sonuçları tabloya ekle
    table = PrettyTable()
    table.field_names = ["Metrik", "Değer"]
    table.add_row(["ROUGE-1", round(rouge_scores["rouge1"], 4)])
    table.add_row(["ROUGE-2", round(rouge_scores["rouge2"], 4)])
    table.add_row(["ROUGE-L", round(rouge_scores["rougeL"], 4)])
    table.add_row(["BLEU", round(bleu_score["bleu"], 4)])
    table.add_row(["METEOR", round(meteor_score["meteor"], 4)])
    table.add_row(["BERTScore Precision", round(bert_precision, 4)])
    table.add_row(["BERTScore Recall", round(bert_recall, 4)])
    table.add_row(["BERTScore F1", round(bert_f1, 4)])
    table.add_row(["Exact Match", round(exact_match_score, 4)])
    table.add_row(["Contains Correct Result", round(correct_result_score, 4)])
    table.add_row(["Fuzzy Match", round(fuzzy_match_avg, 4)])

    # Sonuçları yazdır
    print(table)

    # wandb log
    wandb.log({
        "ROUGE-1": rouge_scores["rouge1"],
        "ROUGE-2": rouge_scores["rouge2"],
        "ROUGE-L": rouge_scores["rougeL"],
        "BLEU": bleu_score["bleu"],
        "METEOR": meteor_score["meteor"],
        "BERTScore Precision": bert_precision,
        "BERTScore Recall": bert_recall,
        "BERTScore F1": bert_f1,
        "Exact Match": exact_match_score,
        "Contains Correct Result": correct_result_score,
        "Fuzzy Match": fuzzy_match_avg
    })

    print("\n✅ Model değerlendirme tamamlandı ve tüm metrikler wandb'a loglandı.")


In [ ]:
train_dataset[0]["text"]

'Sigarayı Bırakmak İçin 9 Şubat Bir Dönüm Noktası Olsun » 08 Olay | En Güncel Artvin Haberleri Artvin İçin Görüyoruz, Artvin İçin Yazıyoruz\nYeşilay Artvin Şube Başkanı Av. Tüncer Başer, “9 Şubat Dünya Sigarayı Bırakma Günü” dolayısıyla önemli açıklamalarda bulundu.\nYeşilay Artvin Şube Başkanı Av. Tüncer Başer dünya genelinde 1,2 milyar insan sigara kullandığına dikkat çekerek tütünden hastalıklar sebebiyle her yıl 6 milyon insanımızı hayatını kaybettiğine dikkat çekti.\nBaşer, sigaranın hem kişiye hem çevresine verdiği zararları hatırlatarak, sigara bağımlıları için 9 Şubat Dünya Sigarayı Bırakma Günü’nün bir dönüm noktası olabileceğini söyleyerek şu açıklamalarda bulundu:\n“Dünya Sağlık Örgütü, dünyada en büyük sağlık sorunun sigara olduğunu açıkladı. İstatistiklere göre sigara içmek dünya çapında bir problem ve tahminen her 3 yetişkinden biri sigara kullanıyor. Sigara, fiziksel, sosyal ve ekonomik bakımdan yıkıcı sonuçlara yol açan ciddi bir bağımlılıktır. Ülkemizde her yıl 100 bin

In [ ]:
#model.gradient_checkpointing_enable()

In [ ]:
torch.autograd.set_detect_anomaly(True)

In [ ]:
#val_dataset = val_dataset.select(range(10100, 11000))

In [ ]:
import math

def get_warmup_steps_from_dataset(dataset_len, batch_size, num_epochs, pct=0.05):
    """
    Dataset bilgisine göre dinamik warmup step sayısı hesaplar.

    Args:
        dataset_len (int): Dataset’teki toplam örnek sayısı.
        batch_size (int): Batch başına örnek sayısı.
        num_epochs (int): Toplam epoch sayısı.
        pct (float): Warmup oranı (0.03 - 0.1 arası önerilir).

    Returns:
        int: Warmup step sayısı.
    """
    steps_per_epoch = math.ceil(dataset_len / batch_size)
    total_steps = steps_per_epoch * num_epochs
    warmup_steps = int(total_steps * pct)
    return warmup_steps


In [ ]:
from transformers import TrainerCallback
import torch
import wandb
import os
import shutil

class GradientCheckCallback(TrainerCallback):
    def on_step_end(self, args, state, control, **kwargs):
        model = kwargs["model"]

        found_problem = False
        for name, param in model.named_parameters():
            if param.grad is not None:
                if torch.isnan(param.grad).any():
                    print(f"🚨 NaN in gradients of {name}")
                    found_problem = True
                if torch.isinf(param.grad).any():
                    print(f"🚨 Inf in gradients of {name}")
                    found_problem = True

        if found_problem:
            print(f"⛔ Problematic gradients detected at step {state.global_step}!")
            
            control.should_training_stop = True  # Eğitimi durdur


        return control


class ManualGradientClipCallback(TrainerCallback):
    def __init__(self, max_grad_norm=1.0):
        self.max_grad_norm = max_grad_norm

    def on_step_end(self, args, state, control, **kwargs):
        model = kwargs["model"]

        # Gradyanları kliple
        total_norm = torch.nn.utils.clip_grad_norm_(
            model.parameters(), self.max_grad_norm
        )

        if torch.isnan(total_norm) or torch.isinf(total_norm):
            print(f"🚨 NaN/Inf gradyan normu! Step: {state.global_step}")
        elif total_norm > self.max_grad_norm:
            print(f"⚠️ Gradyan norm ({total_norm:.2f}) sınırı aştı, kliplendi.")

        return control


class WandbTextGenerationCallback(TrainerCallback):
    def __init__(self, tokenizer, prompt="Ali sabah uyanır ve pencereden dışarı bakar. Hava", log_interval=50, device="cuda"):
        self.tokenizer = tokenizer
        self.prompt = prompt
        self.log_interval = log_interval
        self.device = device
        self.table = wandb.Table(columns=["step", "prompt", "output"])


    def on_step_end(self, args, state, control, **kwargs):
        if state.global_step % self.log_interval == 0 and state.global_step != 0:
            model = kwargs['model'].to(self.device)
            

            # Tokenize prompt and move to correct device + dtype
            input_ids = self.tokenizer.encode(self.prompt, return_tensors="pt").to(self.device)
            #input_ids = input_ids.to(dtype=model_dtype)

            max_new_tokens = 1024
            with torch.no_grad():
                generated_ids = model.generate(
                                            input_ids, 
                                            max_new_tokens=max_new_tokens, 
                                            use_cache=True, 
                                            pad_token_id=tokenizer.pad_token_id,
                                            eos_token_id=tokenizer.eos_token_id,
                                            )
            output_text = self.tokenizer.decode(generated_ids[0], skip_special_tokens=True)

            # Tabloya ekle
            self.table.add_data(state.global_step, self.prompt, output_text)

            # WandB'ye logla
            wandb.log({"text_generation/table": self.table}, step=state.global_step)
            
            print(f"\n🧪 [Step {state.global_step}] Prompt Testi:\n🟢 Prompt: {self.prompt}\n🔵 Output: {output_text}")


class WandbModelSaverCallback(TrainerCallback):
    def __init__(self, save_interval=500):
        self.artifacts = []
        self.save_interval = save_interval

    def on_save(self, args, state, control, **kwargs):
        if state.global_step % self.save_interval != 0:
            return control  # ⛔ Save interval dışında, hiçbir şey yapma

        checkpoint_dir = os.path.join(args.output_dir, f"checkpoint-{state.global_step}")
        artifact_name = f"crispy-checkpoint-{state.global_step}"
        artifact = wandb.Artifact(name=artifact_name, type="model")

        artifact.add_dir(checkpoint_dir)
        wandb.log_artifact(artifact)
        self.artifacts.append(artifact_name)

        # 🧹 Temizlik: WandB staging cache
        staging_dir = os.path.join(os.path.expanduser("~"), ".local", "share", "wandb", "artifacts", "staging")
        try:
            shutil.rmtree(staging_dir)
            print(f"✅ Cleaned WandB staging folder: {staging_dir}")
        except Exception as e:
            print(f"⚠️ Could not clean staging folder: {e}")

        return control


In [ ]:
def tokenize_fn(example):
   
    full_text = example["text"]
    tokenized = tokenizer(
        full_text,
        padding="max_length",
        truncation=True,
        max_length=max_seq_length,
        return_tensors="pt"
    )
    
    tokenized["input_ids"] = tokenized["input_ids"][0]
    tokenized["labels"] = tokenized["input_ids"].clone()
    tokenized["attention_mask"] = tokenized["attention_mask"][0]
    

    return tokenized

train_dataset = train_dataset.map(tokenize_fn, remove_columns=train_dataset.column_names)
val_dataset = val_dataset.map(tokenize_fn, remove_columns=val_dataset.column_names)
test_dataset = test_dataset.map(tokenize_fn, remove_columns=test_dataset.column_names)

In [ ]:
tokenizer.decode(train_dataset[100]["input_ids"]), tokenizer.decode(train_dataset[100]["labels"])

("<s> Michel Jules Arthur Tromont (d.24 Haziran 1937 – ö.9 Temmuz 2018), Belçikalı siyasetçidir. Liberal Reformcu Parti üyesi olarak, 1978'den 1983'e kadar Temsilciler Meclisi'nde (Belçika) milletvekili olarak çalışmış, 1981'den 1983'e kadar Fransa Milli Eğitim Bakanı ve 1977'den 1983'e kadar Quiévrain belediye başkanı olarak görev yapmıştı. Aynı zamanda 1983'ten 2004'e kadar Hainaut kentinin valisi olarak görev yapan Belçikalı siyasetçi Michel Tromont 9 Temmuz 2018'de 81 yaşında ölmüştür. Kaynakça 1937 doğumlular 2018 yılında ölenler Belçikalı siyasetçiler Belçikalı bakanlar Bilgi kutusu bulunmayan makam sahipleri</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad

In [ ]:
num_epochs = 2

In [ ]:
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

args = TrainingArguments(
    # 🚀 Eğitim Temelleri
    num_train_epochs=num_epochs,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=16,
    eval_accumulation_steps=16,
    output_dir="Crispy-330M-V2-Rope-NewTokenizer-JustLanguage",
    seed=3407,
    no_cuda=False,
    use_cpu=False,
    auto_find_batch_size=True,

    # 🧠 Optimizasyon
    optim="adamw_torch_fused",
    learning_rate=0.001,
    weight_decay=0.01,
    adam_beta2=0.95,
    max_grad_norm=1.0,

    # 🌀 Öğrenme Oranı Planlayıcı
    lr_scheduler_type="cosine_with_restarts",
    lr_scheduler_kwargs={"num_cycles": 5},
    warmup_ratio=1.0 / 10,  # num_epochs = 2 ise

    # 🔄 Değerlendirme & Checkpoint
    eval_strategy="steps",
    eval_steps=3000,
    save_steps=50,
    save_total_limit=2,

    # 🧠 Precision Ayarları
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),

    # 📜 Loglama & İzleme
    logging_strategy="steps",
    logging_steps=50,
    #log_level="debug",              # Ana işlem log seviyesi
    #log_level_replica="warning",    # Diğer işlem log seviyesi (dağıtık eğitimde)
    report_to=["wandb"],
    logging_nan_inf_filter=True,

    # 🧹 Bellek ve Checkpointing
    gradient_checkpointing=False,
    torch_empty_cache_steps=50,
)


In [ ]:
from transformers import TrainingArguments, DataCollatorForSeq2Seq, Trainer

trainer = Trainer(
    model = model,
    #tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    #dataset_text_field = "text",
    #max_seq_length = max_seq_length,
    #data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    #dataset_num_proc = 2,
    #packing = False, # Can make training 5x faster for short sequences.
    #callbacks=[wandb_callback],
    #packing=False,
    #remove_unused_columns=True,
    #torch_compile=True,
    callbacks=[WandbTextGenerationCallback(tokenizer=tokenizer), 
               GradientCheckCallback(), 
               ManualGradientClipCallback(), 
               WandbModelSaverCallback(save_interval=250) ],
    args = args
)

In [ ]:
from IPython.display import display

# Canlı izleme için
display(wandb.run)


In [ ]:
trainer.train(
                resume_from_checkpoint=True
            )

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss


In [ ]:
# Test değerlendirmesi
#evaluate_model(model, tokenizer, test_dataset, max_seq_length=max_seq_length)

In [ ]:
# 6. Eğitilmiş Modeli Kaydedin
model.save_pretrained("./Crispy-330M-V2-Rope-NewTokenizer-JustLanguage")
tokenizer.save_pretrained("./Crispy-330M-V2-Rope-NewTokenizer-JustLanguage")

print("Eğitim tamamlandı ve model kaydedildi.")

In [ ]:
import unsloth
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
import torch
from datasets import Dataset, DatasetDict, concatenate_datasets, load_dataset

In [ ]:
from transformers import XLMRobertaTokenizer

# XLM-Roberta tokenizer yükleniyor
tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")
special_tokens_dict = {
    "bos_token": "<s>",
    "eos_token": "<|eot_id|>",
    "additional_special_tokens":  [
        "<|im_start|>", "<|im_end|>",
        "<|system|>", "<|user|>", "<|assistant|>",
        "<|start_header_id|>", "<|end_header_id|>", "<|eot_id|>"
    ]
}


tokenizer.add_special_tokens(special_tokens_dict)

In [ ]:
import torch

# Model ve tokenizer'ını yükle
from transformers import PreTrainedTokenizerFast
from transformers import AutoConfig, AutoModelForCausalLM
from MyLLM.CrispyLLM_RoPE2.modeling_crispy_rope import CrispyLLMConfig, CrispyForCausalLM


# 3. Kayıt (Auto ile kullanabilmek için)
AutoConfig.register("crispy", CrispyLLMConfig)
AutoModelForCausalLM.register(CrispyLLMConfig, CrispyForCausalLM)
model = AutoModelForCausalLM.from_pretrained("./Crispy-330M-V2-Rope-NewTokenizer-JustLanguage/checkpoint-8500" ,  
                                            attn_implementation="flash_attention_2",
                                            trust_remote_code=True,
                                            torch_dtype=torch.bfloat16,
                                            device_map="auto"
      ).cuda().eval()


In [ ]:
""" 
# Sohbet geçmişi
chat_history = ""

# Cevap üretme fonksiyonu
def generate_response(prompt, max_new_tokens=256):
    input_text = chat_history + prompt
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            use_cache=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = output_text[len(input_text):].strip()
    return response

print("🧠 Crispy Chatbot hazır! Çıkmak için Ctrl+C, sıfırlamak için '/reset' yaz.")
print("-" * 50)

# Sonsuz konuşma döngüsü
while True:
    user_input = input("👤 Sen: ")
    
    if user_input.strip().lower() == "/reset":
        chat_history = ""
        print("🔁 Sohbet sıfırlandı.")
        continue

    chat_history += f"👤 Sen: {user_input}\n"
    response = generate_response(f"👤 Sen: {user_input}\n🤖 Crispy:")
    chat_history += f"🤖 Crispy: {response}\n"

    print(f"🤖 Crispy: {response}")
 """

In [ ]:
input_text = """Ali sabah uyanır ve pencereden dışarı bakar. Hava"""

In [ ]:
input_ids  = tokenizer(input_text, padding="max_length", max_length=1024,return_tensors="pt").to(model.device)

with torch.no_grad():
    # Modelden yanıt üret
    generated_ids = model.generate(
        **input_ids, 
        max_new_tokens=1024 ,
        do_sample=False,
        use_cache=True,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        #num_beams=5, 
        no_repeat_ngram_size=3,  
        early_stopping=True,
       # top_k=50,
       # top_p=0.9,
        #temperature=0.9,
    )

# Üretilen token'ları geri metne çevir
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

print(generated_text[len(input_text):])

